In [1]:
# 处理初始数据集paper，提取关键字


class paper(object):
    def __init__(self):
        self.name = []
        self.affiliations = []
data = [paper() for i in range(2500000)]#创建一个类，包含每一篇文章的作者名，机构，年份


In [2]:
filename1 = r"C:\Users\Huawei\Desktop\PY实验\数据处理\paper-washed_affiliations-展开缩写.txt"
def Handling_text(num,str):
    if str[:2] == '#@':#名字提取
        data[num].name = str[3:len(str)-1].split(';')
    if str[:2] == '#t':#年份提取
        data[num].year = str[3:len(str)-1]
        if data[num].year == '':
            data[num].year = '0000'
    if str[:2] == '#o':#机构提取
        data[num].affiliations = str[3:len(str)-1].split(';')
sum = -1
with open(filename1,encoding='utf-8',errors='ignore') as f:
    for line in f.readlines():
        if(line[:6] == '#index'):
            sum = sum+1
        else :
            Handling_text(sum,line)

In [3]:
# 计算hash值
def calculate_hash_number(s):
    hash = 5381
    for c in s:
        hash = (hash*33+ord(c))%147777777
    return hash

In [4]:
def add_linked_list(name):#将名字信息加入链表中，方便查找
    global Linked_P
    hash_number = calculate_hash_number(name)
    Linked_next.append(Linked_head[hash_number])
    Linked_head[hash_number] = Linked_P
    Linked_P += 1

In [5]:
#初始化hash值的位置并创建链表
filename2 = r"C:\Users\Huawei\Desktop\PY实验\数据处理\名字hash表（author_washed）.txt"
hash_answer = []#存储hash表中的内容
Linked_head = [-1 for i in range(150000000)]#创建链表头文件
Linked_next = []#链表next文件
Linked_P = 0
p = 0
with open(filename2,encoding='utf-8') as f2:
    for line in f2.readlines():
        hash_answer.append(line[:len(line)-1].split('*!@$'))
        hash_answer[p][4] = int(hash_answer[p][4])#讲hi转化为int类型
        add_linked_list(hash_answer[p][2])#将名字的信息加入列表中
        p += 1

In [6]:
#找寻名字的编号
import re
from fuzzywuzzy import fuzz
def creat_new_name_data(a,b):#在新编号的名字中寻找是否有匹配信息，如果没有则作为新加入的信息，如果有直接返回之前已经编好的号码
    global P1 
    global P
    tt = calculate_hash_number(data[a].name[b])
    hash_answer.append([str(tt),str(P1),data[a].name[b],data[a].affiliations[b],1])
    add_linked_list(data[a].name[b])
    P1 += 1
    P += 1
    return P-1
def find_capital(s):
    lis = s.split(' ')
    ls1 = []
    for c in lis:
        c.replace('-','')
        if len(re.findall(r'[A-Z]',c)) > 0:
            if c != 'The':
                ls1.append(c)
    s1 = ''.join(ls1)
    return s1
def match_affiliations(s1,s2):     #匹配机构信息，将paper中的作者机构信息用正则拆分成单个的单词去看这些单词在author中的机构中是否出现，
                                   #如果匹配的占比达到一定的阈值，则判定是同一个人，不满足则判定不是同一个人
    s1 = find_capital(s1)
    cont = 0
    owe = s2.split(';')
    maxx = 0
    for i in owe:
        s3 = find_capital(i)
        maxx = max(fuzz.token_set_ratio(s1,s3),maxx)
    return maxx
def find_name_number(t):#寻找名字的编号
    global hi_or_pc
    global the_number_of_same_name
    answer = []
    for j in range(len(data[t].name)):
        tt = calculate_hash_number(data[t].name[j])
        p = Linked_head[tt]
        if p == -1:#如果在原author文件中找不到对应的人，则在新创建的信息中找
            locat = creat_new_name_data(t,j)
            answer.append(hash_answer[locat][1])
            continue
        maxx = 0
        cont = 0
        owe = []
        locat = -1
        while(p != -1):#寻找hash_answer中名字对应的人，并将其下标加入owe列表中
            if data[t].name[j] == hash_answer[p][2]:
                owe.append(p)
            p = Linked_next[p]
        if len(owe) == 0:#如果没有找到名字对应的人，那么就创建一个新的信息
            locat = creat_new_name_data(t,j)
            answer.append(hash_answer[locat][1])
            continue
        if len(owe) == 1 and int(hash_answer[owe[0]][1]) < 1712434:#如果有且仅有一个名字对应的人，那么就直接匹配给这个人
            locat = owe[0]
        if len(owe) > 1:#如果有两个及以上名字对应的人，即出现同名情况，那么需要匹配机构，如果paper中的这个人的机构信息缺失，那么直接把
                        #他匹配给hi或pc最高的那个人。   
            for i in owe:
                if data[t].affiliations[j] != '-':
                    MATCH_RATING = match_affiliations(data[t].affiliations[j],hash_answer[i][3])
                    if MATCH_RATING > maxx and MATCH_RATING > 50:
                        maxx = MATCH_RATING
                        locat = i
                else :
                    if maxx <= hash_answer[i][4]:
                        maxx = hash_answer[i][4]
                        locat = i
            if data[t].affiliations[j] == '-':
                hi_or_pc += 1
        if locat != -1 and len(owe) > 1 :
            the_number_of_same_name += 1
        if locat == -1:
            locat = creat_new_name_data(t,j)
            answer.append(hash_answer[locat][1])
            continue
        if locat > 171433:
            hash_answer[locat][4] += 1
        answer.append(hash_answer[locat][1])
    return answer

d:\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
#找寻名字的编号
from itertools import combinations
ff = open(r"C:\Users\Huawei\Desktop\PY实验\数据处理\编号匹配.txt","w",encoding = 'utf-8')
name_number = []
P = len(hash_answer)
P1 = 1712434
hi_or_pc = 0#记录通过hi或者pc来匹配的人的个数
the_number_of_same_name = 0#记录总同名个数
tt1 = (sum+1)
for i in range(tt1):
    name_number.append(find_name_number(i))
for i in range(tt1):
    for j in range(len(data[i].name)):
        ff.write(str(name_number[i][j])+'*!@$')
    ff.write(data[i].year+'\n')
print(hi_or_pc,' ',the_number_of_same_name)
ff.close()

899413   1986551


In [8]:
#提取合作关系
ff = open(r"C:\Users\Huawei\Desktop\PY实验\数据处理\合作关系提取（按年升序排列）.txt","w",encoding = 'utf-8')
from itertools import combinations
symbiosis = []
for i in range(sum+1):
    if len(data[i].name) == 1:
        continue
    year = data[i].year
    for j in combinations(name_number[i],2):
        lis = list(j)
        if lis[0] > lis[1]:
            lis[0],lis[1] = lis[1],lis[0]
        lis.append(year)
        symbiosis.append(lis)
symbiosis.sort(key = lambda symbiosis:symbiosis[2])
for i in range(len(symbiosis)):
    ff.write(symbiosis[i][0]+'*!@$'+symbiosis[i][1]+'*!@$'+symbiosis[i][2]+'\n')
ff.close()